## Shelby County Transportation Recovery Model

In [ ]:
import json
import pandas as pd
from pyincore import IncoreClient, InventoryDataset, Dataset, DataService
from pyincore.analyses.bridgedamage import BridgeDamage
from pyincore.analyses.transportationrecovery import TransportationRecovery
from pyincore.analyses.transportationrecovery import TransportationRecoveryUtil

## RUN Bridge damage analysis

### Set up input datasets and parameters

In [ ]:
client = IncoreClient()

In [ ]:
hazard_type = "earthquake"
hazard_id = "5b902cb273c3371e1236b36b"

# NBSR bridges
bridge_dataset_id = "5a284f2dc7d30d13bc082040"

# Default Bridge Fragility Mapping on incore-service
mapping_id = "5b47bcce337d4a37755e0cb2"

# Use hazard uncertainty for computing damage
use_hazard_uncertainty = False

# Use liquefaction (LIQ) column of bridges to modify fragility curve
use_liquefaction = False

In [ ]:
# Create bridge damage 
bridge_dmg = BridgeDamage(client)

# Load input datasets
bridge_dmg.load_remote_input_dataset("bridges", bridge_dataset_id)

# Specify the result name
result_name = "bridge_result"

# Set analysis parameters
bridge_dmg.set_parameter("result_name", result_name)
bridge_dmg.set_parameter("mapping_id", mapping_id)
bridge_dmg.set_parameter("hazard_type", hazard_type)
bridge_dmg.set_parameter("hazard_id", hazard_id)
bridge_dmg.set_parameter("num_cpu", 4)

In [ ]:
bridge_dmg.run_analysis()

In [ ]:
bridge_dmg.get_output_datasets()

### damamge result ouptut

In [ ]:
bridge_damage = pd.read_csv('bridge_result.csv')
bridge_damage.head()

## lookup average daily traffic from National Bridge Inventory

In [ ]:
bridges_set = InventoryDataset('cache_data/nbsrbridges/nbsrbridges.shp')
bridges = list(bridges_set.inventory_set)

datasvc = DataService(client)
datasvc.get_dataset_blob("5c5de6c1c5c0e488fc04291b")
TN17 = 'cache_data/TN17/TN17.shp'
ADT = TransportationRecoveryUtil.get_average_daily_traffic(bridges, TN17)

In [ ]:
with open('cache_data/ADT.json', 'w') as f:
    json.dump(ADT, f)

### convert bridge damage output to the format that transportation recovery model accepts

In [ ]:
bridge_damage_value, unrepaired_bridge = TransportationRecoveryUtil.convert_dmg_prob2state("bridge_result.csv")

In [ ]:
with open('cache_data/bridge_damage_value.json', 'w') as f:
    json.dump(bridge_damage_value, f)
with open('cache_data/unrepaired_bridge.json', 'w') as f:
    json.dump(unrepaired_bridge, f)

## Calculate Transportation Recovery

In [ ]:
transportation_recovery = TransportationRecovery(client)

In [ ]:
nodes = "5c5de1dec5c0e488fc0355f7"
transportation_recovery.load_remote_input_dataset("nodes", nodes)

links = "5c5de25ec5c0e488fc035613"
transportation_recovery.load_remote_input_dataset("links", links)

bridges = "5a284f2dc7d30d13bc082040"
transportation_recovery.load_remote_input_dataset('bridges', bridges)

bridge_damage = "5c5ddff0c5c0e488fc0355df"
transportation_recovery.load_remote_input_dataset('bridge_damage_value', bridge_damage)

unrepaired = "5c5de0c5c5c0e488fc0355eb"
transportation_recovery.load_remote_input_dataset('unrepaired_bridge', unrepaired)

ADT_data = "5c5dde00c5c0e488fc032d7f"
transportation_recovery.load_remote_input_dataset('ADT', ADT_data)

In [ ]:
transportation_recovery.set_parameter("num_cpu", 4)
transportation_recovery.set_parameter("pm", 1)
transportation_recovery.set_parameter('ini_num_population', 5)
transportation_recovery.set_parameter("population_size", 3)
transportation_recovery.set_parameter("num_generation", 2)
transportation_recovery.set_parameter("mutation_rate", 0.1)
transportation_recovery.set_parameter("crossover_rate", 1.0)

In [ ]:
transportation_recovery.run_analysis()

In [ ]:
transportation_recovery.get_output_datasets()